In [24]:
import pandas as pd

door_df = pd.read_csv('door_coord_df_single.csv')
door_df.head()

,door,xmin,ymin,xmax,ymax
0,1,208,666,397,1049
1,1,208,666,397,1049
2,1,208,666,397,1049
3,1,208,666,397,1049
4,1,232,661,476,1049


In [25]:
door_df[door_df['door']==2].head()

,door,xmin,ymin,xmax,ymax
28,2,869,396,951,683
29,2,869,396,951,683
30,2,869,396,951,683
31,2,869,396,951,683
32,2,877,397,970,684


In [26]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(door_df, test_size=0.1)

In [27]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=1000,
                           task_type="GPU",
                           devices='0:1')
model.fit(train[['xmin','ymin','xmax','ymax']],
          train[['door']],
          verbose=False)

In [28]:
pred = model.predict(test[['xmin','ymin','xmax','ymax']])

In [29]:
test['pred']=pred

In [30]:
test.head()

,door,xmin,ymin,xmax,ymax,pred
4113,4,1225,268,1249,332,4
2296,5,1306,260,1325,313,5
297,2,911,374,1030,679,2
3890,4,1270,279,1316,392,4
961,4,1219,234,1254,332,4


In [34]:
all_shape=test.shape
all_shape

(549, 6)

In [35]:
good_shape =test[test['door']==test['pred']].shape
good_shape

(523, 6)

In [37]:
good_shape[0]/all_shape[0]

0.9526411657559198

In [16]:
model.save_model("door_predict.cbm")

In [38]:
train2_df=pd.read_csv('door_coord_df2_single.csv')

In [39]:
pred = model.predict(train2_df[['xmin','ymin','xmax','ymax']])

In [40]:
train2_df['pred']=pred
train2_df.head()

,door,xmin,ymin,xmax,ymax,pred
0,1,182,652,311,905,1
1,1,172,646,368,975,1
2,1,179,649,338,907,1
3,1,207,663,354,915,1
4,1,238,680,373,909,1


In [41]:
all2_shape=train2_df.shape
all2_shape

(8144, 6)

In [42]:
good2_shape=train2_df[train2_df['door']==train2_df['pred']].shape
good2_shape

(5772, 6)

In [44]:
good2_shape[0]/all2_shape[0]

0.7087426326129665

In [23]:
37505/58593

0.6400935265304729

In [47]:
for i in range(1,10):
    door_i_df = train2_df[train2_df['door']==i]
    all2_i_shape=door_i_df.shape
    if all2_i_shape[0]==0:
        print(f'Door {i}:empty')
    else:
        good2_i_shape=door_i_df[door_i_df['pred']==i].shape
        print(f'Door {i}:{good2_i_shape[0]/all2_i_shape[0]}({good2_i_shape[0]}/{all2_i_shape[0]})')

Door 1:1.0(15/15)
Door 2:0.7619270346117867(1629/2138)
Door 3:0.591304347826087(272/460)
Door 4:0.8284611141753999(1502/1813)
Door 5:0.8318135764944276(1642/1974)
Door 6:0.6368715083798883(114/179)
Door 7:empty
Door 8:0.36545079146593257(531/1453)
Door 9:0.5982142857142857(67/112)
